In [ ]:
class CustomDualTransform(A.DualTransform):
    def __init__(self, always_apply=False, p=1.0, mask_size=16):
        super(CustomDualTransform, self).__init__(always_apply=always_apply, p=p)
        self.mask_size = mask_size

    def apply(self, img, **params):
        return img

    def apply_to_mask(self, mask, **params):
        return mask

    def get_params(self):
        return {"mask_size": self.mask_size}

    def apply_to_bbox(self, bbox, **params):
        return bbox

    def apply_to_keypoint(self, keypoint, **params):
        return keypoint

    def get_transform_init_args_names(self):
        return ("mask_size",)

    def apply_dual(self, image, mask, mask_size=16, **params):
        """
        Apply Cutout to both image and mask.
        """
        h, w = image.shape[:2]
        y = np.random.randint(h)
        x = np.random.randint(w)
        y1 = np.clip(y - mask_size // 2, 0, h)
        y2 = np.clip(y + mask_size // 2, 0, h)
        x1 = np.clip(x - mask_size // 2, 0, w)
        x2 = np.clip(x + mask_size // 2, 0, w)

        image[y1:y2, x1:x2] = 0  # Cutout

        # Apply the same cutout region to the mask
        mask[y1:y2, x1:x2] = 0

        return image, mask

In [ ]:
def fish(img, distortion_coefficient, color):
    h, w = img.shape[:2]
    
    x, y = np.meshgrid(np.arange(w), np.arange(h))
    xnd, ynd = (2 * x - w) / w, (2 * y - h) / h
    rd = np.sqrt(xnd ** 2 + ynd ** 2)
    
    mask = 1 - distortion_coefficient * (rd ** 2)
    mask[mask == 0] = 1
    xdu, ydu = xnd / mask, ynd / mask
    
    xu, yu = ((xdu + 1) * w / 2).astype(int), ((ydu + 1) * h / 2).astype(int)
    valid_indices = (0 <= xu) & (xu < w) & (0 <= yu) & (yu < h)
    
    dstimg = np.zeros_like(img, dtype=np.uint8)
    dstimg[y[valid_indices], x[valid_indices]] = img[yu[valid_indices], xu[valid_indices]]
    dstimg[~valid_indices] = color
    
    return dstimg.astype(np.uint8)

In [ ]:
class Fisheye(DualTransform):
    def apply(self, img: np.ndarray, distortion_coefficient=1.0, **params) -> np.ndarray:
        if len(img.shape) == 2:
            color=12

        if len(img.shape) == 3 and img.shape[2] == 3:
            color=[0,0,0]

        return fish(img, distortion_coefficient,color)  # 어안 렌즈 효과 적용 이미지 생성
    
    def get_transform_init_args_names(self):
        return ()

In [ ]:
def generate(self, img, flag):

    # image_list = sorted([image for image in os.listdir(src_dir) if image.endswith(".png")])

    # count =0
    # for image in tqdm(image_list):
    src_image = cv2.imread(img, flag)

    if self._F_RAND_FLAG:
        self._generator.rand_f(self._F_RANGE)
    if self._EXT_RAND_FLAG:
        self._generator.rand_ext_params()

    if flag==3:
        result = self._generator.transFromColor(src_image)
    # cv2.imwrite(dst_dir+prefix+image, result1)
    # print("Image", count, "Done!")
    if flag==0:
        result = self._generator.transFromGray(src_image)

    return result

class Fisheye(DualTransform):
    def apply(self, img: np.ndarray, dst_size=[640,640],focal_len=200, ext_param=[10,10,10,0,0,0] **params) -> np.ndarray:
        if len(img.shape) == 2:
            flag=0

        if len(img.shape) == 3 and img.shape[2] == 3:
            flag=3

        DT = FESetsGenerator(dst_size, focal_len)
        DT.set_ext_param_range(ext_param)
        DT.rand_ext_params()
        DT.rand_f()
        return DT.generate(img, flag)

    def get_transform_init_args_names(self):
        return ()

In [ ]:
class Fisheye(DualTransform):
    def apply(self, img: np.ndarray, dst_size=[640,640],focal_len=200, ext_param=[10,10,10,0,0,0] **params) -> np.ndarray:
        if len(img.shape) == 2:
            flag=0

        if len(img.shape) == 3 and img.shape[2] == 3:
            flag=3

        DT = FESetsGenerator(dst_size, focal_len)
        DT.set_ext_param_range(ext_param)
        DT.rand_ext_params()
        DT.rand_f()
        return DT.generate(img, flag)

    def get_transform_init_args_names(self):
        return ()

In [ ]:
class Fisheye(DualTransform):
    def __init__(self, always_apply=False, p=0.5):
        super().__init__(always_apply, p)
        self.dst_size = [1024, 2048]
        self.focal_len = 200
        self.ext_param = [10, 10, 10, 0, 0, 0]
        self.DT = FESetsGenerator(self.dst_size, self.focal_len)
        
    def apply(self, img: np.ndarray, **params) -> np.ndarray:
        if len(img.shape) == 2:
            flag=0
        elif len(img.shape) == 3 and img.shape[2] == 3:
            flag=3

        return self.DT.generate(img, flag)

    def get_transform_init_args_names(self):
        return ()

    def apply_to_mask(self, mask: np.ndarray, **params):
        if len(mask.shape) == 2:
            flag=0
        elif len(mask.shape) == 3 and mask.shape[2] == 3:
            flag=3

        return self.DT.generate(mask, flag)
    
    def update_params(self, params, **kwargs):
        self.DT.set_ext_param_range(self.ext_param)
        self.DT.rand_ext_params()
        self.DT.rand_f()
        return super().update_params(params, **kwargs)

In [ ]:
class FESetsGenerator:

    def __init__(self, dst_shape, focal_len=350):
        self._generator = FishEyeGenerator(focal_len, dst_shape)

        self._F_RAND_FLAG = True
        self._F_RANGE = [200,350]

        self._EXT_RAND_FLAG = True
        self._EXT_PARAM_RANGE = [5, 5, 10, 0.3, 0.3, 0.4]
        self._generator.set_ext_param_range(self._EXT_PARAM_RANGE)

    def set_ext_param_range(self,ext_param):
        for i in range(6):
            self._EXT_PARAM_RANGE[i] = ext_param[i]
        self._generator.set_ext_param_range(self._EXT_PARAM_RANGE)

    def rand_ext_params(self):
        self._EXT_RAND_FLAG = True

    def set_ext_params(self,ext_params):
        self._generator.set_ext_params(ext_params)
        self._EXT_RAND_FLAG = False


    def set_f(self, focal_len):
        self._generator.set_f(focal_len)
        self._F_RAND_FLAG = False

    def rand_f(self, f_range=[200,400]):
        self._F_RANGE = f_range
        self._F_RAND_FLAG = True

    def generate(self, img):
        src_image = img

        if self._F_RAND_FLAG:
            self._generator.rand_f(self._F_RANGE)
        if self._EXT_RAND_FLAG:
            self._generator.rand_ext_params()

        return src_image
# FESetsGenerator 클래스의 인스턴스를 생성합니다.
DT = FESetsGenerator([640, 640], focal_len=350)


class Fisheye(A.DualTransform):
    def __init__(self, **kwargs):
        super().__init__()
        self._generator = FishEyeGenerator(350, [640, 640])

    def apply(self, img: np.ndarray, dst_size=[640,640],focal_len=200, ext_param=[5, 5, 10, 0.3, 0.3, 0.4], **params) -> np.ndarray:
        if len(img.shape) == 2:
            flag=0

        if len(img.shape) == 3 and img.shape[2] == 3:
            flag=3



        DT = FESetsGenerator(dst_size, focal_len)
        DT.set_ext_param_range(ext_param)
        DT.rand_ext_params()
        DT.rand_f()
        src_image= DT.generate(img)

        if flag==3:
            result = self._generator.transFromColor(src_image)
        if flag==0:
            result = self._generator.transFromGray(src_image)
        return result.astype(np.uint8)

    def get_transform_init_args_names(self):
        return ()

In [ ]:
old_param=None
old_focal=None
cnt=0
old_img_path="None"
# dst_size=[1024,2048]

class Fisheye(DualTransform):
    def __init__(self, csv_path="./train_source.csv", **kwargs):
        super().__init__(**kwargs)
        self.csv_path = csv_path
        data = pd.read_csv(csv_path)

        self.img_paths = data['id'].tolist()

        self._generator = FishEyeGenerator(focal_len, dst_shape)

        self._F_RAND_FLAG = False
        self._F_RANGE = [200,350]

        self._EXT_RAND_FLAG = False
        self._EXT_PARAM_RANGE = [5, 5, 10, 0.3, 0.3, 0.4]
        self._generator.set_ext_param_range(self._EXT_PARAM_RANGE)
        # print(self.img_paths)

    def set_ext_param_range(self,ext_param):
        for i in range(6):
            self._EXT_PARAM_RANGE[i] = ext_param[i]
        self._generator.set_ext_param_range(self._EXT_PARAM_RANGE)

    def rand_ext_params(self):
        self._EXT_RAND_FLAG = True

    def set_ext_params(self,ext_params):
        self._generator.set_ext_params(ext_params)
        self._EXT_RAND_FLAG = False

    def set_f(self, focal_len):
        self._generator.set_f(focal_len)
        self._F_RAND_FLAG = False

    def rand_f(self, f_range=[200,400]):
        self._F_RANGE = f_range
        self._F_RAND_FLAG = True

    def generate(self, img, flag, img_id, old_img_id):
        global old_focal, old_param
        src_image = img
        # print("img_id:",img_id, "old_img_id:",old_img_id)

        if img_id==old_img_id:
            self._F_RAND_FLAG=False
            self._EXT_RAND_FLAG=False
        else:
            self._F_RAND_FLAG=True
            self._EXT_RAND_FLAG=True

        # print("f_land flag 상태:", self._F_RAND_FLAG,"ext rand flag:" ,self._EXT_RAND_FLAG)

        if self._F_RAND_FLAG:
            randf=self._generator.rand_f(self._F_RANGE)
            old_focal=randf
        elif not self._F_RAND_FLAG:
            self._generator.set_f(old_focal)

        if self._EXT_RAND_FLAG:
            randext=self._generator.rand_ext_params()
            # print("ext param true")
            # self._generator.print_ext_param()
            old_param=randext
        elif not self._EXT_RAND_FLAG:
            [self._generator._alpha, self._generator._beta, self._generator._theta, self._generator._x_trans, self._generator._y_trans, self._generator._z_trans]=old_param
            # print("ext param false")
            # self._generator.print_ext_param()

        # print(f"old_focal:{old_focal}, old_param:{old_param}")

        if flag==3:
            result = self._generator.transFromColor(src_image)
        if flag==0:
            result = self._generator.transFromGray(src_image)

        return result.astype(np.uint8)
    
    def apply(self, img: np.ndarray, dst_size=[1024, 2048], focal_len=200, ext_param=[5, 5, 10, 0.3, 0.3, 0.4], **params) -> np.ndarray:
        global cnt, old_img_path

        if cnt < len(self.img_paths):
            self.img_path = self.img_paths[cnt]
        else:
            cnt=0

        if self.img_path==old_img_path:
            cnt+=1
     
        DT = FESetsGenerator(dst_size, focal_len)
        DT.set_ext_param_range(ext_param)
        DT.rand_ext_params()
        DT.rand_f()

        if len(img.shape) == 2:
            flag = 0
        elif len(img.shape) == 3 and img.shape[2] == 3:
            flag = 3
        else:
            raise ValueError("Unsupported image shape")
        
        fe=DT.generate(img, flag, self.img_path, old_img_path)

        old_img_path=self.img_path

        return fe

    def get_transform_init_args_names(self):
        return ('csv_path', 'epoch')

In [ ]:
old_param=None
old_focal=None

class FESetsGenerator:
    def __init__(self, dst_shape, focal_len=350):
        self._generator = FishEyeGenerator(focal_len, dst_shape)

        self._F_RAND_FLAG = False
        self._F_RANGE = [200,350]

        self._EXT_RAND_FLAG = False
        self._EXT_PARAM_RANGE = [5, 5, 10, 0.3, 0.3, 0.4]
        self._generator.set_ext_param_range(self._EXT_PARAM_RANGE)

    def set_ext_param_range(self,ext_param):
        for i in range(6):
            self._EXT_PARAM_RANGE[i] = ext_param[i]
        self._generator.set_ext_param_range(self._EXT_PARAM_RANGE)

    def rand_ext_params(self):
        self._EXT_RAND_FLAG = True

    def set_ext_params(self,ext_params):
        self._generator.set_ext_params(ext_params)
        self._EXT_RAND_FLAG = False

    def set_f(self, focal_len):
        self._generator.set_f(focal_len)
        self._F_RAND_FLAG = False

    def rand_f(self, f_range=[200,400]):
        self._F_RANGE = f_range
        self._F_RAND_FLAG = True

    def generate(self, img, flag, img_id, old_img_id):
        global old_focal, old_param
        src_image = img
        # print("img_id:",img_id, "old_img_id:",old_img_id)

        if img_id==old_img_id:
            self._F_RAND_FLAG=False
            self._EXT_RAND_FLAG=False
        else:
            self._F_RAND_FLAG=True
            self._EXT_RAND_FLAG=True

        # print("f_land flag 상태:", self._F_RAND_FLAG,"ext rand flag:" ,self._EXT_RAND_FLAG)

        if self._F_RAND_FLAG:
            randf=self._generator.rand_f(self._F_RANGE)
            old_focal=randf
        elif not self._F_RAND_FLAG:
            self._generator.set_f(old_focal)

        if self._EXT_RAND_FLAG:
            randext=self._generator.rand_ext_params()
            # print("ext param true")
            # self._generator.print_ext_param()
            old_param=randext
        elif not self._EXT_RAND_FLAG:
            [self._generator._alpha, self._generator._beta, self._generator._theta, self._generator._x_trans, self._generator._y_trans, self._generator._z_trans]=old_param
            # print("ext param false")
            # self._generator.print_ext_param()

        # print(f"old_focal:{old_focal}, old_param:{old_param}")

        if flag==3:
            result = self._generator.transFromColor(src_image)
        if flag==0:
            result = self._generator.transFromGray(src_image)

        return result.astype(np.uint8)

In [ ]:
cnt=0
old_img_path="None"

class Fisheye(DualTransform):
    def __init__(self, csv_path="./train_source.csv", **kwargs):
        super().__init__(**kwargs)
        self.csv_path = csv_path
        data = pd.read_csv(csv_path)

        self.img_paths = data['id'].tolist()
        # print(self.img_paths)

    def apply(self, img: np.ndarray, dst_size=[1024, 2048], focal_len=200, ext_param=[5, 5, 10, 0.3, 0.3, 0.4], **params) -> np.ndarray:
        global cnt, old_img_path

        if cnt < len(self.img_paths):
            self.img_path = self.img_paths[cnt]
        else:
            cnt=0

        if self.img_path==old_img_path:
            cnt+=1
     
        DT = FESetsGenerator(dst_size, focal_len)
        DT.set_ext_param_range(ext_param)
        DT.rand_ext_params()
        DT.rand_f()

        if len(img.shape) == 2:
            flag = 0
        elif len(img.shape) == 3 and img.shape[2] == 3:
            flag = 3
        else:
            raise ValueError("Unsupported image shape")
        
        fe=DT.generate(img, flag, self.img_path, old_img_path)

        old_img_path=self.img_path

        return fe

    def get_transform_init_args_names(self):
        return ('csv_path', 'epoch')